# Decision Trees - Heart Disease Classification

**Students:** Album #103569, #103512  
**Dataset:** Heart Disease (Cleveland)  
**Source:** UCI Machine Learning Repository  

---

## Table of Contents
1. [Data Loading and Understanding](#1-data-loading)
2. [Exploratory Data Analysis (EDA)](#2-eda)
3. [Data Preprocessing](#3-preprocessing)
4. [Decision Tree Model](#4-model)
5. [Evaluation](#5-evaluation)
6. [Visualization](#6-visualization)
7. [Conclusions](#7-conclusions)

---
## 1. Data Loading and Understanding

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn import tree

# Set style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (10, 6)

In [ ]:
# Define column names based on UCI repository documentation
column_names = [
    'age',           # age in years
    'sex',           # sex (1 = male; 0 = female)
    'cp',            # chest pain type (1-4)
    'trestbps',      # resting blood pressure (mm Hg)
    'chol',          # serum cholesterol (mg/dl)
    'fbs',           # fasting blood sugar > 120 mg/dl (1 = true; 0 = false)
    'restecg',       # resting ECG results (0-2)
    'thalach',       # maximum heart rate achieved
    'exang',         # exercise induced angina (1 = yes; 0 = no)
    'oldpeak',       # ST depression induced by exercise
    'slope',         # slope of peak exercise ST segment (1-3)
    'ca',            # number of major vessels colored by fluoroscopy (0-3)
    'thal',          # thalassemia (3 = normal; 6 = fixed defect; 7 = reversable defect)
    'target'         # diagnosis (0 = no disease; 1,2,3,4 = disease)
]

# Load data
df = pd.read_csv('data/heart_disease.csv', names=column_names, na_values='?')

print("Dataset Shape:", df.shape)
print("\nFirst 5 rows:")
df.head()

In [ ]:
# Dataset information
print("Dataset Information:")
print("="*50)
df.info()

In [ ]:
# Convert target to binary (0 = no disease, 1 = disease)
# Original: 0 = no disease, 1-4 = presence of disease
df['target'] = (df['target'] > 0).astype(int)

print("Target distribution after conversion:")
print(df['target'].value_counts())
print(f"\nClass balance: {df['target'].value_counts(normalize=True)*100}")

---
## 2. Exploratory Data Analysis (EDA)

### 2.1 Missing Values

In [ ]:
# Check for missing values
missing = df.isnull().sum()
missing_pct = (missing / len(df)) * 100

missing_df = pd.DataFrame({
    'Missing Count': missing,
    'Percentage': missing_pct
})
missing_df = missing_df[missing_df['Missing Count'] > 0].sort_values('Missing Count', ascending=False)

print("Missing Values:")
print(missing_df)

# Visualize missing values
if len(missing_df) > 0:
    plt.figure(figsize=(10, 4))
    plt.bar(missing_df.index, missing_df['Missing Count'])
    plt.title('Missing Values by Feature')
    plt.xlabel('Features')
    plt.ylabel('Count')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()
else:
    print("\nNo missing values found!")

### 2.2 Statistical Summary

In [ ]:
# Statistical summary
print("Statistical Summary:")
print("="*80)
df.describe().T

### 2.3 Distribution Analysis

In [ ]:
# Target distribution
plt.figure(figsize=(8, 5))
target_counts = df['target'].value_counts()
plt.bar(['No Disease', 'Disease'], target_counts.values, color=['green', 'red'], alpha=0.7)
plt.title('Target Distribution (Heart Disease)', fontsize=14, fontweight='bold')
plt.ylabel('Count')
plt.xlabel('Diagnosis')
for i, v in enumerate(target_counts.values):
    plt.text(i, v + 5, str(v), ha='center', fontweight='bold')
plt.tight_layout()
plt.show()

print(f"Class Balance: {target_counts[1]/target_counts[0]:.2f}")

In [ ]:
# Distribution of numerical features
numerical_cols = ['age', 'trestbps', 'chol', 'thalach', 'oldpeak']

fig, axes = plt.subplots(2, 3, figsize=(15, 8))
axes = axes.ravel()

for idx, col in enumerate(numerical_cols):
    axes[idx].hist(df[col].dropna(), bins=20, color='skyblue', edgecolor='black', alpha=0.7)
    axes[idx].set_title(f'Distribution of {col}', fontweight='bold')
    axes[idx].set_xlabel(col)
    axes[idx].set_ylabel('Frequency')
    axes[idx].grid(axis='y', alpha=0.3)

# Hide the last subplot if empty
axes[-1].axis('off')

plt.tight_layout()
plt.show()

In [ ]:
# Box plots for numerical features by target
fig, axes = plt.subplots(2, 3, figsize=(15, 8))
axes = axes.ravel()

for idx, col in enumerate(numerical_cols):
    df.boxplot(column=col, by='target', ax=axes[idx], patch_artist=True)
    axes[idx].set_title(f'{col} by Disease Status')
    axes[idx].set_xlabel('Disease (0=No, 1=Yes)')
    axes[idx].set_ylabel(col)

axes[-1].axis('off')
plt.suptitle('')  # Remove default title
plt.tight_layout()
plt.show()

### 2.4 Correlation Analysis

In [ ]:
# Correlation matrix
plt.figure(figsize=(12, 10))
correlation = df.corr()
sns.heatmap(correlation, annot=True, fmt='.2f', cmap='coolwarm', center=0,
            square=True, linewidths=1, cbar_kws={"shrink": 0.8})
plt.title('Correlation Matrix - Heart Disease Features', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

In [ ]:
# Correlation with target
target_corr = correlation['target'].sort_values(ascending=False)
print("Correlation with Target (Heart Disease):")
print("="*50)
print(target_corr)

# Visualize
plt.figure(figsize=(10, 6))
target_corr[1:].plot(kind='barh', color=['red' if x < 0 else 'green' for x in target_corr[1:]])
plt.title('Feature Correlation with Heart Disease', fontsize=14, fontweight='bold')
plt.xlabel('Correlation Coefficient')
plt.ylabel('Features')
plt.axvline(x=0, color='black', linestyle='--', linewidth=0.8)
plt.tight_layout()
plt.show()

### 2.5 Categorical Features Analysis

In [ ]:
# Categorical features
categorical_cols = ['sex', 'cp', 'fbs', 'restecg', 'exang', 'slope', 'ca', 'thal']

fig, axes = plt.subplots(3, 3, figsize=(15, 12))
axes = axes.ravel()

for idx, col in enumerate(categorical_cols):
    # Create crosstab
    ct = pd.crosstab(df[col], df['target'], normalize='index') * 100
    ct.plot(kind='bar', ax=axes[idx], color=['green', 'red'], alpha=0.7)
    axes[idx].set_title(f'{col} vs Disease', fontweight='bold')
    axes[idx].set_xlabel(col)
    axes[idx].set_ylabel('Percentage')
    axes[idx].legend(['No Disease', 'Disease'])
    axes[idx].grid(axis='y', alpha=0.3)

axes[-1].axis('off')
plt.tight_layout()
plt.show()

---
## 3. Data Preprocessing

In [ ]:
# Handle missing values (drop rows with missing values)
df_clean = df.dropna()
print(f"Original dataset size: {len(df)}")
print(f"Clean dataset size: {len(df_clean)}")
print(f"Rows removed: {len(df) - len(df_clean)}")

In [ ]:
# Separate features and target
X = df_clean.drop('target', axis=1)
y = df_clean['target']

print("Features shape:", X.shape)
print("Target shape:", y.shape)
print("\nFeature names:")
print(list(X.columns))

In [ ]:
# Split data into training and testing sets (80/20)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print("Training set size:", X_train.shape)
print("Testing set size:", X_test.shape)
print("\nTraining set target distribution:")
print(y_train.value_counts())
print("\nTesting set target distribution:")
print(y_test.value_counts())

---
## 4. Decision Tree Model

In [ ]:
# Train decision tree with max_depth=3
clf = DecisionTreeClassifier(max_depth=3, random_state=42)
clf.fit(X_train, y_train)

print("Decision Tree Model trained successfully!")
print(f"Tree depth: {clf.get_depth()}")
print(f"Number of leaves: {clf.get_n_leaves()}")

---
## 5. Evaluation

In [ ]:
# Make predictions
y_train_pred = clf.predict(X_train)
y_test_pred = clf.predict(X_test)

# Calculate accuracy
train_accuracy = accuracy_score(y_train, y_train_pred)
test_accuracy = accuracy_score(y_test, y_test_pred)

print("Model Performance:")
print("="*50)
print(f"Training Accuracy: {train_accuracy:.4f} ({train_accuracy*100:.2f}%)")
print(f"Testing Accuracy: {test_accuracy:.4f} ({test_accuracy*100:.2f}%)")

In [ ]:
# Confusion Matrix
cm = confusion_matrix(y_test, y_test_pred)

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=True,
            xticklabels=['No Disease', 'Disease'],
            yticklabels=['No Disease', 'Disease'])
plt.title('Confusion Matrix - Heart Disease Prediction', fontsize=14, fontweight='bold')
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.tight_layout()
plt.show()

print("\nConfusion Matrix:")
print(cm)

In [ ]:
# Classification Report
print("Classification Report:")
print("="*60)
print(classification_report(y_test, y_test_pred, 
                          target_names=['No Disease', 'Disease']))

In [ ]:
# Feature Importance
feature_importance = pd.DataFrame({
    'feature': X.columns,
    'importance': clf.feature_importances_
}).sort_values('importance', ascending=False)

print("Feature Importance:")
print("="*50)
print(feature_importance)

# Visualize
plt.figure(figsize=(10, 6))
plt.barh(feature_importance['feature'], feature_importance['importance'], color='steelblue')
plt.xlabel('Importance')
plt.ylabel('Features')
plt.title('Feature Importance in Decision Tree', fontsize=14, fontweight='bold')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

### 5.1 Testing Different max_depth Values

In [ ]:
# Test different max_depth values to avoid overfitting
depths = range(1, 11)
train_scores = []
test_scores = []

for depth in depths:
    clf_temp = DecisionTreeClassifier(max_depth=depth, random_state=42)
    clf_temp.fit(X_train, y_train)
    
    train_scores.append(clf_temp.score(X_train, y_train))
    test_scores.append(clf_temp.score(X_test, y_test))

# Plot results
plt.figure(figsize=(10, 6))
plt.plot(depths, train_scores, marker='o', label='Training Accuracy', linewidth=2)
plt.plot(depths, test_scores, marker='s', label='Testing Accuracy', linewidth=2)
plt.xlabel('Max Depth')
plt.ylabel('Accuracy')
plt.title('Model Performance vs Tree Depth', fontsize=14, fontweight='bold')
plt.legend()
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

# Find best depth
best_depth = depths[np.argmax(test_scores)]
print(f"\nBest max_depth: {best_depth}")
print(f"Best test accuracy: {max(test_scores):.4f}")

---
## 6. Visualization

In [ ]:
# Visualize the decision tree
plt.figure(figsize=(20, 10))
tree.plot_tree(clf, 
               feature_names=X.columns,
               class_names=['No Disease', 'Disease'],
               filled=True,
               rounded=True,
               fontsize=10)
plt.title('Decision Tree for Heart Disease Classification', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.savefig('tree_visualization.png', dpi=300, bbox_inches='tight')
plt.show()

print("Tree visualization saved as 'tree_visualization.png'")

In [ ]:
# Alternative: Export tree structure as text
from sklearn.tree import export_text

tree_rules = export_text(clf, feature_names=list(X.columns))
print("Decision Tree Rules:")
print("="*60)
print(tree_rules)

---
## 7. Conclusions

### Key Findings:

1. **Dataset Overview:**
   - 303 samples with 13 features
   - Binary classification: heart disease presence (yes/no)
   - Relatively balanced classes

2. **EDA Insights:**
   - Strong correlations found between certain features and target
   - Most important features: cp (chest pain), thalach (max heart rate), ca (vessels colored)
   - Some missing values handled by removal

3. **Model Performance:**
   - Decision Tree with max_depth=3 achieves good accuracy
   - No significant overfitting observed
   - Feature importance analysis reveals key predictors

4. **Recommendations:**
   - Consider ensemble methods for improved performance
   - Feature engineering could enhance predictions
   - Cross-validation for more robust evaluation

### Next Steps:
- Tune hyperparameters for optimal performance
- Try different algorithms (Random Forest, SVM, etc.)
- Collect more data to improve generalization